In [ ]:
import pandas as pd
import json
import os

# ===== 파일 경로 확인 =====
# 다운로드 받은 폴더 경로를 입력하세요
data_path = './korean_dialogue_summary/'  # 실제 경로로 변경

# 폴더 내 파일 목록 확인
print("폴더 내 파일들:")
for root, dirs, files in os.walk(data_path):
    for file in files[:10]:  # 처음 10개만
        print(f"  - {file}")

print("\n파일 개수:", len(files))

In [ ]:
# ===== 파일 하나 열어보기 =====
# 첫 번째 파일 경로 (실제 파일명으로 변경)
sample_file = './korean_dialogue_summary/train/sample.json'  # 예시

# JSON 파일인 경우
try:
    with open(sample_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    print("JSON 데이터 구조:")
    print(json.dumps(data[0] if isinstance(data, list) else data, 
                     indent=2, ensure_ascii=False)[:500])
except:
    print("JSON 형식이 아니거나 파일 경로가 다릅니다.")

# CSV 파일인 경우
try:
    df = pd.read_csv(sample_file, encoding='utf-8')
    print("\nCSV 데이터 구조:")
    print(df.head())
    print("\n컬럼:", df.columns.tolist())
except:
    print("CSV 형식이 아닙니다.")

In [ ]:
def load_aihub_json_data(data_dir):
    """AI Hub JSON 형식 데이터 로드"""
    
    dialogues = []
    summaries = []
    
    # 모든 JSON 파일 찾기
    json_files = []
    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if file.endswith('.json'):
                json_files.append(os.path.join(root, file))
    
    print(f"총 {len(json_files)}개의 JSON 파일 발견")
    
    # 각 파일 읽기
    for json_file in tqdm(json_files):
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            # 데이터 구조에 따라 수정 필요!
            # 보통 이런 형식:
            # data['dialogue'] 또는 data['conversation']
            # data['summary'] 또는 data['abstractive']
            
            if isinstance(data, list):
                for item in data:
                    dialogue = item.get('dialogue', item.get('conversation', ''))
                    summary = item.get('summary', item.get('abstractive', ''))
                    
                    if dialogue and summary:
                        dialogues.append(dialogue)
                        summaries.append(summary)
            else:
                dialogue = data.get('dialogue', data.get('conversation', ''))
                summary = data.get('summary', data.get('abstractive', ''))
                
                if dialogue and summary:
                    dialogues.append(dialogue)
                    summaries.append(summary)
                    
        except Exception as e:
            print(f"에러 발생 ({json_file}): {e}")
            continue
    
    print(f"\n로드 완료: {len(dialogues):,}개 샘플")
    return dialogues, summaries

# 사용
train_dialogues, train_summaries = load_aihub_json_data('./korean_dialogue_summary/train/')
test_dialogues, test_summaries = load_aihub_json_data('./korean_dialogue_summary/test/')

In [ ]:
def load_aihub_csv_data(csv_path):
    """AI Hub CSV 형식 데이터 로드"""
    
    df = pd.read_csv(csv_path, encoding='utf-8')
    
    print("컬럼:", df.columns.tolist())
    print("샘플 수:", len(df))
    
    # ===== 컬럼명 확인 후 수정 =====
    dialogues = df['dialogue'].tolist()  # 실제 컬럼명으로
    summaries = df['summary'].tolist()   # 실제 컬럼명으로
    # =================================
    
    return dialogues, summaries

# 사용
train_dialogues, train_summaries = load_aihub_csv_data('./korean_dialogue_summary/train.csv')

In [ ]:
print("="*60)
print("한국어 대화 요약 데이터 로드 완료")
print("="*60)

print(f"\n학습 데이터: {len(train_dialogues):,}개")
if test_dialogues:
    print(f"테스트 데이터: {len(test_dialogues):,}개")

print("\n샘플 3개:")
for i in range(min(3, len(train_dialogues))):
    print(f"\n[샘플 {i+1}]")
    print(f"대화 길이: {len(train_dialogues[i])} 글자")
    print(f"대화: {train_dialogues[i][:150]}...")
    print(f"요약: {train_summaries[i]}")

# 길이 통계
dialogue_lengths = [len(d) for d in train_dialogues]
summary_lengths = [len(s) for s in train_summaries]

print(f"\n평균 대화 길이: {np.mean(dialogue_lengths):.1f} 글자")
print(f"평균 요약 길이: {np.mean(summary_lengths):.1f} 글자")
print(f"압축 비율: {np.mean(summary_lengths)/np.mean(dialogue_lengths):.2%}")

In [ ]:
# ===== 한국어 대화 요약 데이터 로드 =====

# 방법 선택 (위에서 확인한 형식에 맞게)
# JSON 형식인 경우:
train_dialogues, train_summaries = load_aihub_json_data('./korean_dialogue_summary/train/')
test_dialogues, test_summaries = load_aihub_json_data('./korean_dialogue_summary/test/')

# 또는 CSV 형식인 경우:
# train_dialogues, train_summaries = load_aihub_csv_data('./korean_dialogue_summary/train.csv')
# test_dialogues, test_summaries = load_aihub_csv_data('./korean_dialogue_summary/test.csv')

print(f"데이터 로드 완료!")
print(f"학습: {len(train_dialogues):,}개")
print(f"테스트: {len(test_dialogues):,}개")